In [1]:
val malware_list_rdd = sc.wholeTextFiles("s3a://ml-workflow-data/security/Malware_Dataset/Malware_Classification_DataSet/*.asm")

val malware_df = spark.createDataFrame(malware_list_rdd).toDF("File_name", "Text").
withColumn("FileName",split(col("File_name"),"/").
getItem(6)).withColumn("FileName", regexp_replace($"FileName", ".asm" , "") )

val fileName_df= spark.read.format("csv").option("inferSchema", "true").
option("header", "true").
load("s3a://ml-workflow-data/security/Malware_Dataset/Malware_Classification_DataSet/Training_Label_Classification_SmallSet.csv")

val mergedDf = malware_df.join(fileName_df, Seq("FileName"), "inner").
withColumnRenamed("File_name","FileLocation").
withColumnRenamed("Text","RawText").
withColumn("Index",monotonically_increasing_id)


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6,application_1597507779640_0018,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

malware_list_rdd: org.apache.spark.rdd.RDD[(String, String)] = s3a://ml-workflow-data/security/Malware_Dataset/Malware_Classification_DataSet/*.asm MapPartitionsRDD[1] at wholeTextFiles at <console>:25
malware_df: org.apache.spark.sql.DataFrame = [File_name: string, Text: string ... 1 more field]
fileName_df: org.apache.spark.sql.DataFrame = [FileName: string, Label: int]
mergedDf: org.apache.spark.sql.DataFrame = [FileName: string, FileLocation: string ... 3 more fields]


In [2]:
mergedDf.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+-----+-----+
|            FileName|        FileLocation|             RawText|Label|Index|
+--------------------+--------------------+--------------------+-----+-----+
|01kcPWA9K2BOxQeS5Rju|s3a://ml-workflow...|HEADER:10000000		...|    1|    0|
|04EjIdbPV5e1XroFOpiN|s3a://ml-workflow...|HEADER:4DF30000		...|    1|    1|
|05EeG39MTRrI6VY21DPd|s3a://ml-workflow...|HEADER:10000000		...|    1|    2|
|05rJTUWYAKNegBk2wE8X|s3a://ml-workflow...|HEADER:7DF00000		...|    1|    3|
|0AnoOZDNbPXIr2MRBSCJ|s3a://ml-workflow...|HEADER:10000000		...|    1|    4|
|0AwWs42SUQ19mI7eDcTC|s3a://ml-workflow...|HEADER:675A0000		...|    1|    5|
|0DNVFKwYlcjO7bTfJ5p1|s3a://ml-workflow...|HEADER:10000000		...|    1|    6|
|0DqUX5rkg3IbMY6BLGCE|s3a://ml-workflow...|HEADER:10000000		...|    1|    7|
|0G4hwobLuAzvl1PWYfmd|s3a://ml-workflow...|HEADER:00400000		...|    1|    8|
|0H63jydvIahOVqgx5Kfo|s3a://ml-workflow...|HEADER:00400000		...|    1|    9|

In [6]:
mergedDf.filter(mergedDf("Label")===2).show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+-----+-----+
|            FileName|        FileLocation|             RawText|Label|Index|
+--------------------+--------------------+--------------------+-----+-----+
|0P6tEopzr2mIhkuOKiCD|s3a://ml-workflow...|.text:00401000			...|    2|   16|
|0PREAmkFZjrftD3LiwOV|s3a://ml-workflow...|.text:00401000			...|    2|   17|
|0RvUg8rntNxyDHPJeEYG|s3a://ml-workflow...|.text:00401000			...|    2|   20|
|0SUAyQWoTeZb8qmIPMK3|s3a://ml-workflow...|.zenc:00401000			...|    2|   24|
|0TBj7glMRJScN9GmOzyi|s3a://ml-workflow...|.text:00401000			...|    2|   27|
+--------------------+--------------------+--------------------+-----+-----+
only showing top 5 rows



In [ ]:
val max_count=mergedDf.count()

In [28]:
for(i <- 20 until max_count by 10){
    val start=i-20
    val end=i
    println(start + "-" + end)
    val df_temp=mergedDf.filter('Index > start && 'Index <= end)
    df_temp.write.mode("append").format("parquet").option("compression","snappy").saveAsTable("malware_tmp_1")
    
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid status code '400' from http://ip-192-168-2-139.us-west-2.compute.internal:8998/sessions/4/statements/28 with error payload: {"msg":"requirement failed: Session isn't active."}


In [28]:
for(i <- 20 until max_count by 10){
    val start=i-20
    val end=i
    println(start + "-" + end)
    val df_temp=mergedDf.filter('Index > start && 'Index <= end)
    df_temp.write.mode("append").format("parquet").option("compression","snappy").saveAsTable("malware_tmp_2")
    
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid status code '400' from http://ip-192-168-2-139.us-west-2.compute.internal:8998/sessions/4/statements/28 with error payload: {"msg":"requirement failed: Session isn't active."}


In [ ]:
import org.apache.spark.ml.feature.{CountVectorizer, CountVectorizerModel}
import org.apache.spark.ml.feature.{RegexTokenizer, Tokenizer}
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.ml.{Pipeline, PipelineModel}

val tokenizer=new Tokenizer().setInputCol("RawText").setOutputCol("Words")
val cnt = new  CountVectorizer().setInputCol("Words").setOutputCol("Features").setVocabSize(10)


val pipeline = new Pipeline().setStages(Array(tokenizer, cnt))
val model = pipeline.fit(mergedDf)
val df_temp = model.transform(mergedDf)
val df_transformed=df_temp.drop("RawText")



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
for (i <- 1 until 9){
    df_transformed.filter(df_transformed("Label")===i).write.mode("append").format("parquet").option("compression","snappy").saveAsTable("malware_tmp_2")
    
}


In [3]:
for (j <- 1 until 10 by1){
    for(i <- 20 until 821 by 10){
    val start=i-20
    val end=i
    println("j="+j+" "+start + "-" + end)
   // val df_temp=df_transformed.filter('Index > start && 'Index <= end)
    //df_temp.write.mode("append").format("parquet").option("compression","snappy").saveAsTable("malware_tmp_2")
    
}
}
    


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

j=1 0-20
j=1 10-30
j=1 20-40
j=1 30-50
j=1 40-60
j=1 50-70
j=1 60-80
j=1 70-90
j=1 80-100
j=1 90-110
j=1 100-120
j=1 110-130
j=1 120-140
j=1 130-150
j=1 140-160
j=1 150-170
j=1 160-180
j=1 170-190
j=1 180-200
j=1 190-210
j=1 200-220
j=1 210-230
j=1 220-240
j=1 230-250
j=1 240-260
j=1 250-270
j=1 260-280
j=1 270-290
j=1 280-300
j=1 290-310
j=1 300-320
j=1 310-330
j=1 320-340
j=1 330-350
j=1 340-360
j=1 350-370
j=1 360-380
j=1 370-390
j=1 380-400
j=1 390-410
j=1 400-420
j=1 410-430
j=1 420-440
j=1 430-450
j=1 440-460
j=1 450-470
j=1 460-480
j=1 470-490
j=1 480-500
j=1 490-510
j=1 500-520
j=1 510-530
j=1 520-540
j=1 530-550
j=1 540-560
j=1 550-570
j=1 560-580
j=1 570-590
j=1 580-600
j=1 590-610
j=1 600-620
j=1 610-630
j=1 620-640
j=1 630-650
j=1 640-660
j=1 650-670
j=1 660-680
j=1 670-690
j=1 680-700
j=1 690-710
j=1 700-720
j=1 710-730
j=1 720-740
j=1 730-750
j=1 740-760
j=1 750-770
j=1 760-780
j=1 770-790
j=1 780-800
j=1 790-810
j=1 800-820
j=2 0-20
j=2 10-30
j=2 20-40
j=2 30-50
j=2 40-6

In [3]:
spark.sql("select count(*) from  malware_tmp_1 group by Label").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------+
|Label|count(1)|
+-----+--------+
|    1|     140|
|    6|      44|
|    3|     194|
|    8|       6|
|    5|       6|
|    2|     182|
|    7|      14|
|    9|      18|
+-----+--------+

